In [2]:
import pandas as pd
import json
import pytz
from prompt_toolkit.utils import to_str

In [3]:
areas_df_dict = {
    'f_hertz': pd.read_csv(
        '../data/raw/smard/fileshare/market_data/Realisierte_Erzeugung_50Hertz_202301010000_202501010000_Viertelstunde.csv',
        sep=';'
    ),
    'amprion': pd.read_csv(
        '../data/raw/smard/fileshare/market_data/Realisierte_Erzeugung_Amprion_202301010000_202501010000_Viertelstunde.csv',
        sep=';'
    ),
    'tennet': pd.read_csv(
        '../data/raw/smard/fileshare/market_data/Realisierte_Erzeugung_TenneT_202301010000_202501010000_Viertelstunde.csv',
        sep=';'
    ),
    'transnet_bw': pd.read_csv(
        '../data/raw/smard/fileshare/market_data/Realisierte_Erzeugung_TransnetBW_202301010000_202501010000_Viertelstunde.csv',
        sep=';'
    )
}

In [13]:
len(areas_df_dict['f_hertz'])

70176

In [14]:
with open('/home/aklei/Dropbox/Masterarbeit/Daten/SMARD/smard_api_amprion.json', 'r') as file:
    data = json.load(file)

df = pd.DataFrame(data['series'], columns=['timestamp', 'value_amprion'])
df['timestamp'] = pd.to_datetime(df['timestamp'], unit='ms')
df.set_index('timestamp', inplace=True)
df.index = df.index.tz_localize('UTC').tz_convert('Europe/Berlin')

print(df.tail())
print("\nLength time series: ")
print(len(df))


                           value_amprion
timestamp                               
2023-01-01 22:45:00+01:00         401.00
2023-01-01 23:00:00+01:00         401.25
2023-01-01 23:15:00+01:00         400.25
2023-01-01 23:30:00+01:00         409.75
2023-01-01 23:45:00+01:00         405.75

Length time series: 
672


In [21]:
df_tennet = areas_df_dict['tennet']
df_tennet.columns

Index(['Datum von', 'Datum bis', 'Biomasse [MWh] Originalauflösungen',
       'Wasserkraft [MWh] Originalauflösungen',
       'Wind Offshore [MWh] Originalauflösungen',
       'Wind Onshore [MWh] Originalauflösungen',
       'Photovoltaik [MWh] Originalauflösungen',
       'Sonstige Erneuerbare [MWh] Originalauflösungen',
       'Kernenergie [MWh] Originalauflösungen',
       'Braunkohle [MWh] Originalauflösungen',
       'Steinkohle [MWh] Originalauflösungen',
       'Erdgas [MWh] Originalauflösungen',
       'Pumpspeicher [MWh] Originalauflösungen',
       'Sonstige Konventionelle [MWh] Originalauflösungen'],
      dtype='object')

In [57]:
df_tennet_conventional = df_tennet[[
    'Datum von',
    'Datum bis',
    'Braunkohle [MWh] Originalauflösungen',
    'Steinkohle [MWh] Originalauflösungen',
    'Erdgas [MWh] Originalauflösungen',
    'Sonstige Konventionelle [MWh] Originalauflösungen'
]].copy()

for col in df_tennet_conventional.columns:
    if col != 'Datum von' and col != 'Datum bis':
        df_tennet_conventional[col] = (
            df_tennet_conventional[col]
            .astype(str)
            .str.replace(".", "", regex=False)
            .str.replace(',', '.', regex=False)
        )

        df_tennet_conventional[col] = pd.to_numeric(df_tennet_conventional[col], errors='coerce')

In [58]:
df_tennet_conventional['Braunkohle [MWh] Originalauflösungen'].head()

0    1.75
1    1.75
2    1.75
3    1.75
4    1.75
Name: Braunkohle [MWh] Originalauflösungen, dtype: float64

In [59]:
df_tennet_conventional['Sum Conventional Generation'] = df_tennet_conventional.sum(axis=1, numeric_only=True)
df_tennet_conventional.head()

,Datum von,Datum bis,Braunkohle [MWh] Originalauflösungen,Steinkohle [MWh] Originalauflösungen,Erdgas [MWh] Originalauflösungen,Sonstige Konventionelle [MWh] Originalauflösungen,Sum Conventional Generation
0,01.01.2023 00:00,01.01.2023 00:15,1.75,91.25,119.75,70.0,282.75
1,01.01.2023 00:15,01.01.2023 00:30,1.75,91.25,119.75,70.0,282.75
2,01.01.2023 00:30,01.01.2023 00:45,1.75,91.25,119.75,70.0,282.75
3,01.01.2023 00:45,01.01.2023 01:00,1.75,91.25,119.75,70.0,282.75
4,01.01.2023 01:00,01.01.2023 01:15,1.75,90.50,119.00,70.0,281.25
